# Тестовое задание 2
**Яндекс.Еда осуществляет доставку еды из ресторанов. При этом у каждого ресторана есть зона, в рамках которой осуществляется доставка. Зона представляет собой полигон (заданы координаты его вершин). Пользователь в зависимости от своего местоположения (координат точки) видит разное количество доступных ресторанов. Нам важно, чтобы у каждого пользователя было достаточное количество ресторанов для выбора. Задача заключается в том, чтобы для каждого пользователя посчитать доступное ему количество ресторанов.**

Формат результата: 

id, number_of_places_available

1, 2

2, 19

3, 0 

# Библиотеки

In [ ]:
pip install geopandas

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from itertools import chain
from shapely import wkt

In [ ]:
!apt install libspatialindex-dev
!pip install rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


# Точки
user_coordinates.csv (примерно 300 тыс. строк, user_id – идентификатор пользователя)

In [ ]:
user_coordinates = pd.DataFrame({'user_id':[1,2,3,4,5],
                                 'loc_lat': [55.737564, 56.234564, 55.234578, 55.74702, 55.75078626429166],
                                 'loc_lon' : [37.345186, 37.234590, 36.295745, 37.78707, 37.78239462812255]})

In [ ]:
user_coordinates

,user_id,loc_lat,loc_lon
0,1,55.737564,37.345186
1,2,56.234564,37.234590
2,3,55.234578,36.295745
3,4,55.747020,37.787070
4,5,55.750786,37.782395


Для проверки добавила в user_coordinates еще 2х пользователей. Пользователь 4- с координатами одной из вершин первого полигона. 5 - с координатами центра первого полигона.

Преобразовала координаты в кортежи:

In [ ]:
user_coordinates['lat_long'] = user_coordinates[['loc_lat', 'loc_lon']].apply(tuple, axis=1)

Сделала кортежи точками с координатами:

In [ ]:
user_coordinates['point'] = user_coordinates['lat_long'].apply(lambda x: Point(x))

Преобразовала в в геодатафрейм:

In [ ]:
user_coordinates = gpd.GeoDataFrame(user_coordinates, geometry='point')

In [ ]:
user_coordinates

,user_id,loc_lat,loc_lon,lat_long,point
0,1,55.737564,37.345186,"(55.737564, 37.345186)",POINT (55.73756 37.34519)
1,2,56.234564,37.234590,"(56.234564, 37.23459)",POINT (56.23456 37.23459)
2,3,55.234578,36.295745,"(55.234578, 36.295745)",POINT (55.23458 36.29574)
3,4,55.747020,37.787070,"(55.74702, 37.78707)",POINT (55.74702 37.78707)
4,5,55.750786,37.782395,"(55.75078626429166, 37.78239462812255)",POINT (55.75079 37.78239)


In [ ]:
user_coordinates.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   user_id   5 non-null      int64   
 1   loc_lat   5 non-null      float64 
 2   loc_lon   5 non-null      float64 
 3   lat_long  5 non-null      object  
 4   point     5 non-null      geometry
dtypes: float64(2), geometry(1), int64(1), object(1)
memory usage: 328.0+ bytes


# Полигоны
place_zone_coordinates.csv (примерно 500 тыс. строк,
place_id – идентификатор ресторана, point_number – порядковый номер вершины полигона)

In [ ]:
place_zone_coordinates = pd.DataFrame({'place_id':[1,1,1,1,2,2,2,2],
                                       'loc_lat': [55.747022,55.751713,55.753878,55.751031,55.803885,55.808677,55.809763,55.810840],
                                       'loc_lon' : [37.787073,37.784328,37.777638,37.779351,37.458311,37.464054,37.461314, 37.458654],
                                       'point_number' : [0,1,2,3,0,1,2,3]})

In [ ]:
place_zone_coordinates

,place_id,loc_lat,loc_lon,point_number
0,1,55.747022,37.787073,0
1,1,55.751713,37.784328,1
2,1,55.753878,37.777638,2
3,1,55.751031,37.779351,3
4,2,55.803885,37.458311,0
5,2,55.808677,37.464054,1
6,2,55.809763,37.461314,2
7,2,55.810840,37.458654,3


In [ ]:
place_zone_coordinates['lat_long'] = place_zone_coordinates[['loc_lat', 'loc_lon']].apply(tuple, axis=1)

In [ ]:
df = place_zone_coordinates.groupby(['place_id'])['lat_long'].agg(list).reset_index()

Датафрейм с координатами полигонов:

In [ ]:
df

,place_id,lat_long
0,1,"[(55.747022, 37.787073), (55.751713, 37.784328..."
1,2,"[(55.803885, 37.458311), (55.808677, 37.464054..."


In [ ]:
df['poly'] = df['lat_long'].apply(lambda x: Polygon(x))

In [ ]:
df = gpd.GeoDataFrame(df, geometry='poly')

In [ ]:
df

,place_id,lat_long,poly
0,1,"[(55.747022, 37.787073), (55.751713, 37.784328...","POLYGON ((55.74702 37.78707, 55.75171 37.78433..."
1,2,"[(55.803885, 37.458311), (55.808677, 37.464054...","POLYGON ((55.80389 37.45831, 55.80868 37.46405..."


In [ ]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   place_id  2 non-null      int64   
 1   lat_long  2 non-null      object  
 2   poly      2 non-null      geometry
dtypes: geometry(1), int64(1), object(1)
memory usage: 176.0+ bytes


# Проверка вхождения точки в полигон

In [ ]:
user_coordinates

,user_id,loc_lat,loc_lon,lat_long,point
0,1,55.737564,37.345186,"(55.737564, 37.345186)",POINT (55.73756 37.34519)
1,2,56.234564,37.234590,"(56.234564, 37.23459)",POINT (56.23456 37.23459)
2,3,55.234578,36.295745,"(55.234578, 36.295745)",POINT (55.23458 36.29574)
3,4,55.747020,37.787070,"(55.74702, 37.78707)",POINT (55.74702 37.78707)
4,5,55.750786,37.782395,"(55.75078626429166, 37.78239462812255)",POINT (55.75079 37.78239)


In [ ]:
df

,place_id,lat_long,poly
0,1,"[(55.747022, 37.787073), (55.751713, 37.784328...","POLYGON ((55.74702 37.78707, 55.75171 37.78433..."
1,2,"[(55.803885, 37.458311), (55.808677, 37.464054...","POLYGON ((55.80389 37.45831, 55.80868 37.46405..."


Выделила отдельно id пользователя и его координаты и id полигона с координатами.

In [ ]:
point_name = user_coordinates[['user_id', 'point']]
polygon_name = df[['place_id', 'poly']]

Объединяю с помощью sjoin

In [ ]:
poly_with_point = gpd.sjoin(point_name, polygon_name, how="inner", op='within')

Вот эта точка входит в первый полигон:

In [ ]:
poly_with_point

,user_id,point,index_right,place_id
4,5,POINT (55.75079 37.78239),0,1


In [ ]:
poly_with_point[['user_id', 'place_id']]

,user_id,place_id
4,5,1


Поличилось, что точки, координаты которых заданы изначально в задании не входят ни в первый, ни во второй полигон. Вершина тоже не входит, а вот точка с координатами центра полигона попадает в него.

Проверяю вручную в
центр и  вершину:

In [ ]:
user_coordinates.iloc[4]['point'].within(df.iloc[0]['poly'])

True

In [ ]:
df.iloc[0]['poly'].contains(user_coordinates.iloc[3]['point'])

False

# Проверяю просто на данных без привязки к датафреймам

In [ ]:
# Create Point objects
p1 = Point(55.737564, 37.345186)
p2 = Point(56.234564, 37.23459)
p3 = Point(55.234578, 36.295745)
p4 = Point(55.747022, 37.787073)                  # это вершина полигона 
p5 = Point(55.75078626429166, 37.78239462812255)  # это центр полигона 

# Create a Polygon
coords = [(55.747022, 37.787073),
 (55.751713, 37.784328),
 (55.753878, 37.777638),
 (55.751031, 37.779351)]
poly = Polygon(coords)

In [ ]:
print(poly.centroid)

POINT (55.75078626429166 37.78239462812255)


In [ ]:
# центр входит
p5.within(poly)

True

In [ ]:
# вершина не входит
p4.within(poly)

False